In [2]:
import os
import glob
import numpy as np
from tifffile import imread, imsave
from tqdm import tqdm_notebook as tqdm
from skimage.exposure import histogram
import matplotlib.pyplot as plt

In [6]:
working_dir = '/media/share6/Justin/'# '/data/datasets/ventricle_segmentation'
original_dir = os.path.join(working_dir, 'zika')

syto_paths = glob.glob(os.path.join(original_dir, 'syto*.tif'))
syto_paths.sort()

syto_paths

['/media/share6/Justin/zika/syto_zika3.tif',
 '/media/share6/Justin/zika/syto_zika4.tif']

In [7]:
percentile = 99.7

for path in tqdm(syto_paths, total=len(syto_paths)):
    
    # Load image
    img = imread(path)
    
    # Compute histogram
    freq, centers = histogram(img)
    
    # Find intensity of 99.7 percentile
    total = freq.sum()
    cdf = np.cumsum(freq)
    target = total * (percentile / 100)
    abs_diff = np.abs(cdf - target)
    idx = np.where(abs_diff == abs_diff.min())[0]
    max_val = centers[idx][0]
    
    # Scale image
    preprocessed = img / max_val
    
    # Save image
    output_name = os.path.join(os.path.basename(path).split('.')[0] + '_preprocessed.tif')
    output_path = os.path.join(original_dir, output_name)
    imsave(output_path, preprocessed, compress=1)
    